In [3]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException

In [4]:

def signal_handler(signal, frame, driver, datos_peliculas):
    print("Proceso interrumpido. Guardando datos...")
    df_resultados_iniciales = pd.DataFrame(datos_peliculas)
    df_resultados_iniciales.to_csv('datos_iniciales.csv', index=False)
    print("Datos guardados en 'datos_iniciales.csv'.")
    guardar_datos(datos_peliculas)  #  definir esta función
    driver.close()
    sys.exit(0)
    
def guardar_datos(datos_peliculas):
   # Define el  archivo donde vamos a guardar los datos.
    ruta_archivo = 'fase-3-datos-actor@s-rach.csv'
    
    # Convierte el diccionario a DataFrame
    df_nuevos_datos = pd.DataFrame(datos_peliculas)
    
    if os.path.exists(ruta_archivo):
        # Si el archivo ya existe, carga los datos existentes y los junta.
        df_existentes = pd.read_csv(ruta_archivo)
        df_resultante = pd.concat([df_existentes, df_nuevos_datos], ignore_index=True)
    else:
        # Si el archivo no existe, utiliza los nuevos datos directamente
        df_resultante = df_nuevos_datos
    
    # Guardamos el DataFrame resultante en el archivo, sobrescribiendo el archivo existente
    df_resultante.to_csv(ruta_archivo, index=False)
    
    print(f"Datos guardados o actualizados exitosamente en {ruta_archivo}.")

#Comienza la funcion de extracción de imdb
def get_data(driver,xpath):
    try:
        data = driver.find_element("xpath", xpath).text
    except:
        data = "N/A"
    return data

def extraer_datos_actores(nombre_archivo):
    # He cambiado el string por la variable nombre_archivo
    df_pelis = pd.read_csv(nombre_archivo)

    df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)
    datos_actores_total = []
    
    driver = webdriver.Chrome()
    driver.maximize_window()

    signal.signal(signal.SIGINT, lambda sig, frame: signal_handler(sig, frame, driver, datos_peliculas))  #Esto es para poder interrumpir la función.

    for id_imdb in df_pelis["ID de la Película"][201:205]:
        try:
            url = f"https://www.imdb.com/title/{id_imdb}/"
            driver.get(url)

            sleep(2)
            try:
                driver.find_element(
                    "xpath", '//*[@id="__next"]/div/div/div[2]/div/button[2]').click()
            except NoSuchElementException:
                pass

            sleep(2)  # verificar si este sleep es util o no
            driver.find_element(
                "xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/ul/li[1]/a').click()

            sleep(2)

            lista_actores = []

            # Busco todos los actores de la película
            actores = driver.find_elements(
                'class name', 'cast_list')[0].find_elements('tag name', 'tr')
            
            # aquí cuento cuantos actores hay en la lista. Le resto 1 porque el primer elemento es el encabezado de la tabla y no tiene datos
            numero_maximo_actores = 10 if len(
                actores)-1 > 10 else len(actores)-1

            nombre_peli = get_data(driver, '//*[@id="main"]/div[1]/div[1]/div/div[1]/h3/a')

            for i in range(2, numero_maximo_actores):
                try:

                    sleep(2)

                    driver.find_element(
                        "xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
                    sleep(3)

                    nombre_actxr = get_data(
                        # Cambiado por la función
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span')

                    # Cambiado por la función
                    conocido_por1 = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/a')

                    # Cambiado por la función
                    conocido_por2 = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[2]/div[2]/div[1]/a')
                    conocido_por = [
                        conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]

                    # Cambiado por la función
                    fecha_nacimiento = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[2]/section/aside/div/span[2]')
                    year_nacimiento = fecha_nacimiento.split(
                        " ")[-1:][0] if fecha_nacimiento != "N/A" else "N/A"

                    # Cambiado por la función
                    profesiones = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul')
                    profesiones_string = ", ".join(
                        profesiones.split("\n")) if profesiones != "N/A" else "N/A"

                    # Cambiado por la función
                    premios = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span')

                    info_actores = (nombre_peli, nombre_actxr, conocido_por,
                                    year_nacimiento, profesiones_string, premios)
                    lista_actores.append(info_actores)
                    

                except NoSuchElementException:
                    print(f"\033[95mNo se encontró el actor número {i-1}\033[0m")
                    pass

                finally:
                    driver.back()

            print("Datos de los actores:")
            print(lista_actores)
            
            datos_actores_total.extend(lista_actores)
        except Exception as e:
            print(f"Error al procesor la película {nombre_peli} con ID de IMDb {id_imdb}: {e}")

    driver.quit()
    return datos_actores_total

In [ ]:
df_peliculas = pd.read_csv("peliculas_api.csv")

In [ ]:
# Llamamos a  la función con nuestro archivo de pelis que sacamos de la API.
extraccion = extraer_datos_actores("peliculas_api.csv")

In [ ]:
df_extraccion = df = pd.DataFrame(extraccion, columns=["nombre_peli", "nombre_actxr", "conocido_por", "year_nacimiento", "profesiones_string", "premios"])

In [ ]:
df_extraccion

In [ ]:
df_extraccion.to_csv("actorxs_vio.csv")